## Fisher Score- Chisquare Test For Feature Selection
Compute chi-squared stats between each non-negative feature and class.

This score should be used to evaluate categorical variables in a classification task.
This score can be used to select the n_features features with the highest values for the test chi-squared statistic from X, which must contain only non-negative features such as booleans or frequencies (e.g., term counts in document classification), relative to the classes.

Recall that the chi-square test measures dependence between stochastic variables, so using this function “weeds out” the features that are the most likely to be independent of class and therefore irrelevant for classification. The Chi Square statistic is commonly used for testing relationships between categorical variables.

It compares the observed distribution of the different classes of target Y among the different categories of the feature, against the expected distribution of the target classes, regardless of the feature categories.

In [94]:
import seaborn as sns

**importing the titanic dataset present in the seaborn library**


In [95]:
df = sns.load_dataset("titanic")

In [96]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [97]:
df.shape

(891, 15)

In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


**Considering few columns to perform chi2 test**

In [99]:
df  = df[[ 'sex','embarked','alone','pclass','survived']]
df.head()

,sex,embarked,alone,pclass,survived
0,male,S,False,3,0
1,female,C,False,1,1
2,female,S,True,3,1
3,female,S,False,1,1
4,male,S,True,3,0


### Performing label encoding on few of the columns 

In [100]:
import numpy as np

In [101]:
df['sex'] = np.where(df['sex']=="male",1,0)
df.head()

,sex,embarked,alone,pclass,survived
0,1,S,False,3,0
1,0,C,False,1,1
2,0,S,True,3,1
3,0,S,False,1,1
4,1,S,True,3,0


In [102]:
label = {k: i for i, k in enumerate(df['embarked'].unique(), 0)}
df['embarked']=df['embarked'].map(label)
df.head()

,sex,embarked,alone,pclass,survived
0,1,0,False,3,0
1,0,1,False,1,1
2,0,0,True,3,1
3,0,0,False,1,1
4,1,0,True,3,0


In [103]:
df['alone']=np.where(df['alone']==True,1,0)
df.head()

,sex,embarked,alone,pclass,survived
0,1,0,0,3,0
1,0,1,0,1,1
2,0,0,1,3,1
3,0,0,0,1,1
4,1,0,1,3,0


In [104]:
df['embarked'].unique()

array([0, 1, 2, 3], dtype=int64)

In [105]:
X = df.drop(labels="survived",axis =1)
Y = df['survived']


In [106]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3)

In [107]:
from sklearn.feature_selection import chi2
f_p_values = chi2(x_train,y_train)

**F score value should be higher in order to select a feature**

In [108]:
import pandas as pd
f_values = f_p_values[0]
f_values = pd.Series(f_values)
f_values.index = x_train.columns
f_values.sort_values(ascending = False)

sex         68.689331
pclass      21.222565
alone       10.647529
embarked    10.314637
dtype: float64

In [109]:
p_values = f_p_values[1]
p_values = pd.Series(p_values)
p_values.index = x_train.columns
p_values.sort_values(ascending = True)

sex         1.152628e-16
pclass      4.089217e-06
alone       1.102181e-03
embarked    1.319793e-03
dtype: float64

**Therefore, from the above F score and P value we know which feature is more important : 
sex > pclass > alone > embarked**